# User Example 1: coherence

__Description & purpose__: TODO 

__Author(s)__: Alastair Graham, Dusan Figala

__Date created__: 2024-11-08

__Date last modified__: 2024-11-08

__Licence__: This file is licensed under [Creative Commons Attribution-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-sa/4.0/).  Any included code is released using the [BSD-2-Clause](https://www.tldrlegal.com/license/bsd-2-clause-license-freebsd) license.


<span style="font-size:0.75em;">
Copyright (c) , All rights reserved.</span>

<span style="font-size:0.75em;">
Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:</span>

<span style="font-size:0.75em;">
Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution. THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.</span>

# Background

A requirement to process coherence from Sentinel 1 data was articulated.  

# The process


In [1]:
import logging

logging.basicConfig(level=logging.INFO)
import json
from itertools import combinations

import eo_tools
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as riox
import shapely
from eo_tools.S1.process import process_insar
from eodag import EODataAccessGateway
from shapely.geometry import shape

dag = EODataAccessGateway()
# make sure cop_dataspace will be used
dag.set_preferred_provider("cop_dataspace")
logging.basicConfig(level=logging.INFO)

INFO:eodag.config:Loading user configuration from: /home/al/.config/eodag/eodag.yml
INFO:eodag.core:usgs: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:aws_eos: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:meteoblue: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:hydroweb_next: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:wekeo: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:creodias_s3: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:Locations configuration loaded from /home/al/.config/eodag/locations.yml


In [2]:
# Run search

bbox = {
    "lonmin": 0.08905898091569497,
    "latmin": 52.15527412683906,
    "lonmax": 0.9565339502005088,
    "latmax": 52.69722175598818,
}
shp = shapely.box(bbox["lonmin"], bbox["latmin"], bbox["lonmax"], bbox["latmax"])

search_criteria = {
    "productType": "S1_SAR_SLC",
    "start": "2023-09-03",
    "end": "2023-09-17",
    "geom": bbox,
}

results, _ = dag.search(**search_criteria)

# Find overlaps
data = []
for item in results:
    id = item.properties["id"]
    geom = shape(item.geometry)
    data.append({"id": id, "geometry": geom})

gdf = gpd.GeoDataFrame(data, crs="EPSG:4326")  # Assuming WGS84

# 98% overlap
threshold = 0.98

overlaps = []
for (idx1, row1), (idx2, row2) in combinations(gdf.iterrows(), 2):
    intersection = row1["geometry"].intersection(row2["geometry"])
    if not intersection.is_empty:
        # Calculate overlap ratio as the area of intersection divided by the area of the smaller polygon
        overlap_ratio = intersection.area / min(
            row1["geometry"].area, row2["geometry"].area
        )
        if overlap_ratio >= threshold:
            overlaps.append((row1["id"], row2["id"], overlap_ratio))

overlap_ids = [entry[:-1] for entry in overlaps]
overlap_ids

INFO:eodag.core:Searching product type 'S1_SAR_SLC' on provider: cop_dataspace
INFO:eodag.search.qssearch:Sending search request: http://catalogue.dataspace.copernicus.eu/resto/api/collections/Sentinel1/search.json?startDate=2023-09-03&completionDate=2023-09-17&geometry=POLYGON ((0.0891 52.1553, 0.0891 52.6972, 0.9565 52.6972, 0.9565 52.1553, 0.0891 52.1553))&productType=SLC&maxRecords=20&page=1&exactCount=1
INFO:eodag.core:Found 7 result(s) on provider 'cop_dataspace'


[('S1A_IW_SLC__1SDV_20230904T174209_20230904T174236_050181_060A2D_E88F',
  'S1A_IW_SLC__1SDV_20230916T174209_20230916T174236_050356_061016_8033'),
 ('S1A_IW_SLC__1SDV_20230916T174145_20230916T174212_050356_061016_9D13',
  'S1A_IW_SLC__1SDV_20230904T174144_20230904T174211_050181_060A2D_46E8')]

In [3]:
# Set download dirs
data_dir = "/home/al/Downloads/eotools"

ids = [
    "S1A_IW_SLC__1SDV_20241016T174206_20241016T174233_056131_06DE72_1B0C",
    "S1A_IW_SLC__1SDV_20241028T174206_20241028T174233_056306_06E564_F046"
]
    #"S1A_IW_SLC__1SDV_20230904T174209_20230904T174236_050181_060A2D_E88F",
    #"S1A_IW_SLC__1SDV_20230916T174209_20230916T174236_050356_061016_8033",
#]

primary_dir = f"{data_dir}/{ids[0]}.zip"
secondary_dir = f"{data_dir}/{ids[1]}.zip"
outputs_prefix = f"{data_dir}/res/test-full-processor"

In [4]:
# AOI around Thetford
file_aoi = f"{data_dir}/thetfordaoi.geojson"
shp2 = gpd.read_file(file_aoi).geometry[0]

In [5]:
eo_tools.util.explore_products(results, shp2)

# Generating the workflow

# Additional information

Airbus ONEATLAS DEVELOPER PORTAL https://www.geoapi-airbusds.com/index.html

High Resolution SAR data: https://intelligence.airbus.com/imagery/our-optical-and-radar-satellite-imagery/radar-constellation/
Interferometry case studies: https://intelligence.airbus.com/search/?q=interferometry